In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
%matplotlib tk
import pyxem as pxm
import pymatgen as pmg
import numpy as np
import hyperspy.api as hs

In [72]:
fp = '/home/pc494/Documents/tutorials/pycrystem_examples/nanowire_precession.hdf5'

Nanowire down (110) - nearest peaks are 111 then 002

In [73]:
dp = pxm.ElectronDiffraction(pxm.load(fp))
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]));

A Jupyter Widget

In [74]:
#dp.plot(cmap='gray_r')

First you build your structure

In [76]:
Ga = pmg.Element("Ga")
As = pmg.Element("As")
lattice = pmg.Lattice.cubic(5.6535)

In [78]:
structure = pmg.Structure.from_spacegroup("F23",lattice, [Ga,As], [[0, 0, 0],[0.25,0.25,0.25]])

In [77]:
recip_d_111 = np.sqrt((3/5.6535**2))

Next you set up your microscope paramaters in order to generate the templates

In [79]:
edc = pxm.DiffractionGenerator(300, 5e-2)
diff_gen = pxm.DiffractionLibraryGenerator(edc)

Now you specify what is going into the simulation microscope, in terms of phase and oreintation

In [80]:
from pyxem.utils.sim_utils import equispaced_so3_grid

In [81]:
rot_list = equispaced_so3_grid(90,180,90,5)
struc_lib = dict()
struc_lib["A"] = (structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=recip_d_111/11.4,
                                            reciprocal_radius=0.75,
                                            representation='euler')

In [82]:
dp = dp.inav[10:12,10:12]
dp.set_calibration((recip_d_111/11.4))

With your library built you are ready to tackle the experimental data

In [83]:
from pyxem.indexation_generator import IndexationGenerator
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate()

A Jupyter Widget

In [84]:
bf = match_results.inav[1,0].data[0]

In [85]:
pattern = library["A"][bf[1],bf[2],bf[3]]

In [86]:
pattern.indices

array([[-4.,  0.,  0.],
       [-3., -1., -1.],
       [-3.,  1.,  1.],
       [-2., -2., -2.],
       [-2.,  0.,  0.],
       [-2.,  2.,  2.],
       [-1., -1., -1.],
       [-1.,  1.,  1.],
       [ 0., -2., -2.],
       [ 0.,  0.,  0.],
       [ 0.,  2.,  2.],
       [ 1., -1., -1.],
       [ 1.,  1.,  1.],
       [ 2., -2., -2.],
       [ 2.,  0.,  0.],
       [ 2.,  2.,  2.],
       [ 3., -1., -1.],
       [ 3.,  1.,  1.],
       [ 4.,  0.,  0.]])

In [87]:
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

peaks= match_results.map(peaks_from_best_template,phase=["A"],library=library,inplace=False)

A Jupyter Widget

In [88]:
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=False)